In [135]:
from collections import defaultdict, deque
import random
import math
import time

import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)

    #Definiowanie krawedzi i przekazanie do metody addEdge
    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")


    
    #Wersja dla 1 NN
    def NN(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        visited.add(startNode)
        distance = 0
        currentNode = startNode

        while len(path) < len(self.adjList) + 1:
            closest_distance = float('inf')
            closest_neighbor = None
            
            for neighbour in self.adjList[currentNode]:
                if neighbour not in visited:
                    # Obliczanie odlegosci do poszczegolnych sasiadow
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    # Wybranie najblizszego sasiada
                    if dist < closest_distance:
                        closest_distance = dist
                        closest_neighbor = neighbour

                # Powrot do miasta 0
                elif len(path) == len(self.adjList) and neighbour == 0:
                    path.append(neighbour)
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    distance += dist
                    continue
            # Dodanie sasiada do sciezki o ile to mozliwe
            if closest_neighbor is not None:
                path.append(closest_neighbor)
                distance += closest_distance
                visited.add(closest_neighbor)
                currentNode = closest_neighbor
            else:
                # print(f'{currentNode} has no neighoburs')
                break
        if len(path) == len(cities):
            return path, distance
        else:
            print('\nDidnt find')
            return None, 0

    def NN2(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        dist = []
        visited.add(startNode)
        distance = 0
        currentNode = []
        neighbours = []
        currentNode.append(startNode)

        while len(path) < len(self.adjList) + 1:
            #print(f'Path len:{len(path)}, {len(self.adjList)}')
            closest_distance = float('inf')
            closest_neighbor = None
            
            for i in range(0, len(currentNode)):
                # print(f'\033[91mCurrent node: {currentNode}, {i}, Len: {len(currentNode)}\033[0m')
                for neighbour in self.adjList[currentNode[i]]:
                    if neighbour not in visited:
                        if sym == 'sym':
                            dist.append([self.calculate_distance(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        elif sym == 'asym':
                            dist.append([self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        # print(f'added: {neighbour}')

                    
                    elif len(path) == len(self.adjList) and neighbour == 0:
                        # print(f'Dodatkowe, {path}')
                        path.append(neighbour)
                        if sym == 'sym':
                            dist.append(self.calculate_distance(cities[currentNode[i]], cities[neighbour]))
                        elif sym == 'asym':
                            dist.append(self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]))
                        distance = np.float64(distance)
                        distance += dist
                        break
                    # print(f'\033[92mNieghbours: {neighbours}\033[0m')
                    


                # Warunek sprawdzajacy czy aktualne miasto ma sasiada oraz czy ma mozliwosc (jest wiecej niz 1 mozliwy sasiad)
                if len(neighbours) == 0 and len(path) < len(self.adjList):
                    print(f'\nBad path: {path}')
                    # print(len(path), len(self.adjList), len(path) != len(self.adjList))
                    print(f'Error, current neighbours: {self.adjList[currentNode[i]]}')
                    print(f'Last neighbours: {self.adjList[path[len(path) - 2]]}')
                    # Jezeli waruenk jest spelniony, wtedy zamieniane jest miasto na drugiego sasiada poprzedniego miasta
                    path.remove(currentNode[0])
                    visited.remove(currentNode[0])
                    path.append(currentNode[1])
                    visited.add(currentNode[1])
                    print(f'Path: {path}, visited: {visited}')
                neighbours.clear()

                if len(dist) != 0:
                    # print(f'is not None')
                    break

            if len(path) < len(self.adjList):
                # print(dist, type(dist))
                # Sortowanie po odleglosci od sasiada
                dist = sorted(dist, key=lambda row: row[0])

                # print(f'Distance: {dist}')

                closest_distance = dist[0][0]
                # if len(path) < len(self.adjList) - 1:
                #     closest_neighbor = [dist[0][1], dist[1][1]]

                # Dodanie jednego lub dwoch sasiadow 
                if len(dist) >= 2:
                    closest_neighbor = [dist[0][1], dist[1][1]]
                else:
                    closest_neighbor = [dist[0][1]]
                dist.clear()
            
            # Dodanie sasiada do sciezki o ile to mozliwe
            if closest_neighbor is not None:
                path.append(closest_neighbor[0])
                # print(f'Path: {path}')
                distance += closest_distance
                visited.add(closest_neighbor[0])
                currentNode = closest_neighbor
            else:
                # print(f'{currentNode} has no neighoburs')
                break
        return path, distance

In [145]:
cities = []
for i in range(0, 6):
    cities.append(City())
    # print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

#print(graph.sym_matrix)
#print(graph.matrix_80)
# print(graph.asym100_matrix)
print(graph.asym80_matrix)

#graph.createEdge(graph.sym_matrix)
# graph.createEdge(graph.matrix_80)
# graph.createEdge(graph.asym100_matrix)
graph.createEdge(graph.asym80_matrix)


start_time = time.perf_counter()
NN_path, NN_distance = graph.NN(0, 'asym', cities)
print(f'Path: {NN_path}, distance: {NN_distance}')
end_time = time.perf_counter()
print(f'NN time: {(end_time - start_time) * 100}')

start_time = time.perf_counter()
NN_path, NN_distance = graph.NN2(0, 'asym', cities)
print(f'\nPath: {NN_path}, distance: {NN_distance}')
end_time = time.perf_counter()
print(f'kNN, k = 2 time: {(end_time - start_time) * 100}')

[[0 1 1 1 1]
 [2 0 2 0 0]
 [2 1 0 1 0]
 [2 1 2 0 2]
 [2 1 2 0 0]]

Didnt find
Path: None, distance: 0
NN time: 0.013529999978345586

Bad path: [0, 2, 3, 1]
Error, current neighbours: [0, 2]
Last neighbours: [0, 1, 2, 4]
Path: [0, 2, 3, 4], visited: {0, 2, 3, 4}

Path: [0, 2, 3, 4, 1, 0], distance: [550.30745203]
kNN, k = 2 time: 0.01903000002130284
